In [43]:
import uproot
import awkward as ak
import json

import sys
import os
import argparse
import gc

import numpy as np
import pandas as pd

# Añade el directorio padre al sys.path
sys.path.append(os.path.abspath("/eos/home-d/dcostasr/SWAN_projects/2025_data"))
sys.path.append('/eos/home-d/dcostasr/SWAN_projects/software/hipy')

from wcte.brbtools import sort_run_files, get_part_files, select_good_parts, get_files_from_part
from wcte.brbtools import concat_dfs, df_extend, filter_times_and_relocate_columns, df_mpmt_sumCharge
from wcte.mapping  import id_names

In [44]:
run        = 1570
map        = id_names
run_files  = sort_run_files(f"/eos/experiment/wcte/data/2025_commissioning/offline_data/{run}/WCTE_offline_R{run}S*P*.root")
part_files = get_part_files(run_files)
good_parts = select_good_parts(part_files, run_files)

Selecting Good Parts: 100%|██████████| 118/118 [00:15<00:00,  7.41it/s]


In [ ]:
df_all, hits_df = concat_dfs(good_parts[:10], run_files, map)

Creating DataFrames For Each Part:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
df_good_times, good_events = filter_times_and_relocate_columns(df_all)

In [ ]:
hits_df_good_events = hits_df[hits_df["evt"].isin(good_events)]

In [ ]:
hits_df_good_events

,evt,card,channel,charge,time
0,0,25,6,160.0,1992.774048
1,0,25,10,112.0,1993.678101
2,0,103,6,88.0,1367.580811
3,0,111,5,77.0,1742.446655
4,0,36,7,306.0,1713.531372
...,...,...,...,...,...
14960244,25422,130,13,244.0,2500.473511
14960245,25422,130,16,261.0,2504.637085
14960246,25422,130,15,9.0,2774.776733
14960247,25422,130,15,401.0,4094.923584


In [ ]:
def create_df_from_file(files, time_correction_map):
    """
    Input the information created with get_files_from_part and a dict {(card, channel): correction}.
    Applies time corrections before creating the DataFrame.
    """
    cards        = files[0]
    channels     = files[1]
    charges      = files[2]
    hit_times    = files[3]

    nevents = len(cards)
    evts    = np.arange(nevents)
    nhits   = [len(cards[ievt]) for ievt in evts]
    evt_column = np.repeat(evts, nhits)

    xcards    = ak.flatten(cards)
    xchannels = ak.flatten(channels)
    xcharges  = ak.flatten(charges)
    xtimes    = ak.flatten(hit_times)

    # Calcular la corrección usando NumPy para evitar apply
    xcorrections = np.array([time_correction_map.get((int(c), int(ch)), 0) for c, ch in zip(xcards, xchannels)])
    xtimes_corrected = xtimes + xcorrections

    # Crear el DataFrame con los tiempos corregidos
    df = pd.DataFrame({
        'evt': evt_column,
        'card': xcards,
        'channel': xchannels,
        'charge': xcharges,
        'time': xtimes,
        'time_corrected': xtimes_corrected
    })

    return df


In [ ]:
with open('/eos/home-d/dcostasr/SWAN_projects/NiCf/offline_trigger/mmc_map_R1609.json') as f:
    mcc_map = json.load(f)

d = {}
for k,v in zip(mcc_map.keys(), mcc_map.values()):
    card, channel = [int(i) for i in str(int(k)/100).split(".")]
    d[(card, channel)] = v

mcc_map = d

In [ ]:
files = get_files_from_part(good_parts[0], run_files)

In [ ]:
df = create_df_from_file(files, mcc_map)

In [ ]:
df

,evt,card,channel,charge,time,time_corrected
0,0,25,6,160.0,1992.774048,2254.399403
1,0,25,10,112.0,1993.678101,1993.678101
2,0,103,6,88.0,1367.580811,1119.556308
3,0,111,5,77.0,1742.446655,1753.357417
4,0,36,7,306.0,1713.531372,1718.520188
...,...,...,...,...,...,...
1521059,2518,131,0,1755.0,2406.633667,2406.633667
1521060,2518,131,14,74.0,2469.806396,2469.806396
1521061,2518,131,13,256.0,2485.728516,2485.728516
1521062,2518,131,15,71.0,2497.652588,2497.652588


In [42]:
del df
gc.collect()

6452